<a href="https://colab.research.google.com/github/auromaa-2004/Breast-Cancer-Detection_DL/blob/main/Breast_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [17]:
dataset = load_breast_cancer()
x = dataset.data
y = dataset.target

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x.shape)
print(x_train.shape)
print(x_test.shape)

(569, 30)
(455, 30)
(114, 30)


In [19]:
scaler = StandardScaler()   #zero mean, unit variance

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [20]:
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

In [21]:
class NeuralNet(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [22]:
input_size = x_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [23]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [24]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(x_train)
  loss = criterion(outputs, y_train.view(-1,1))
  loss.backward()
  optimizer.step()

  # claculate accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum()
    accuracy = correct/y_train.size(0)

  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")

Epoch [10/100], Loss : 0.5757, Accuracy: 89.01%
Epoch [20/100], Loss : 0.4513, Accuracy: 92.53%
Epoch [30/100], Loss : 0.3598, Accuracy: 92.75%
Epoch [40/100], Loss : 0.2911, Accuracy: 93.63%
Epoch [50/100], Loss : 0.2401, Accuracy: 93.85%
Epoch [60/100], Loss : 0.2022, Accuracy: 94.73%
Epoch [70/100], Loss : 0.1739, Accuracy: 95.60%
Epoch [80/100], Loss : 0.1523, Accuracy: 96.04%
Epoch [90/100], Loss : 0.1357, Accuracy: 96.70%
Epoch [100/100], Loss : 0.1227, Accuracy: 97.14%


In [26]:
# evaluation on training set
model.eval()
with torch.no_grad():
  outputs = model(x_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1,1)).float().sum()
  accuracy = correct/y_train.size(0)
  print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")

Accuracy on training data: 97.14%


In [27]:
# evaluation on test set
model.eval()
with torch.no_grad():
  outputs = model(x_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1,1)).float().sum()
  accuracy = correct/y_test.size(0)
  print(f"Accuracy on test data: {accuracy.item() * 100:.2f}%")

Accuracy on test data: 98.25%
